In [8]:
!pip install gradio

  Using cached gradio-3.34.0-py3-none-any.whl (20.0 MB)
  Using cached pydub-0.25.1-py2.py3-none-any.whl (32 kB)
  Using cached orjson-3.9.7-cp37-none-win_amd64.whl (134 kB)
  Using cached python_multipart-0.0.8-py3-none-any.whl (22 kB)
  Using cached pygments-2.17.2-py3-none-any.whl (1.2 MB)
  Using cached websockets-11.0.3-cp37-cp37m-win_amd64.whl (124 kB)
  Using cached mdit_py_plugins-0.3.3-py3-none-any.whl (50 kB)
  Using cached altair-5.0.1-py3-none-any.whl (471 kB)
  Using cached pydantic-2.5.3-py3-none-any.whl (381 kB)
  Using cached aiofiles-23.2.1-py3-none-any.whl (15 kB)
  Using cached httpx-0.24.1-py3-none-any.whl (75 kB)
  Using cached semantic_version-2.10.0-py2.py3-none-any.whl (15 kB)
  Using cached fastapi-0.103.2-py3-none-any.whl (66 kB)
  Using cached gradio_client-0.2.6-py3-none-any.whl (288 kB)
  Using cached aiohttp-3.8.6-cp37-cp37m-win_amd64.whl (326 kB)
  Using cached uvicorn-0.22.0-py3-none-any.whl (58 kB)
  Using cached markdown_it_py-2.2.0-py3-none-any.whl (8

ERROR: spyder 4.0.1 requires pyqt5<5.13; python_version >= "3", which is not installed.
ERROR: spyder 4.0.1 requires pyqtwebengine<5.13; python_version >= "3", which is not installed.


In [2]:
!pip install --upgrade python==3.9

ERROR: Could not find a version that satisfies the requirement python==3.9 (from versions: none)
ERROR: No matching distribution found for python==3.9


In [8]:
# !pip install keyring==23.11.0

In [9]:
import yfinance as yf
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import matplotlib.pyplot as plt
from matplotlib import style
import mysql.connector
import mplfinance as mpf
import warnings
import gradio as gr
import pyotp
import qrcode
import io
import bcrypt
from PIL import Image

warnings.filterwarnings('ignore')

def get_history_data(symbol, start, end):
    data = yf.download(symbol, start, end)
    return data

def download_from_yahoo(symbol):
    start = "2020-01-01"
    end = "2024-08-26"
    data = get_history_data(symbol, start, end)
    data.reset_index(inplace=True)
    data.columns = ['date', 'open', 'high', 'low', 'close', 'adj_close', 'volume']
    data['symbol'] = symbol
    data['date'] = data['date'].astype(str)
    
    new_order = ['symbol', 'date', 'open', 'high', 'low', 'close', 'adj_close', 'volume']
    data = data[new_order]
    return data

def roll_back_days(time_str):
    time_format = '%Y-%m-%d'
    time_obj = datetime.strptime(time_str, time_format)
    time_obj -= timedelta(days=30)
    new_time_str = time_obj.strftime(time_format)
    return new_time_str

def read_from_db(symbol, start, end):
    start = roll_back_days(start)
    connection = mysql.connector.connect(
        host='127.0.0.1',       # MySQL服务器地址
        port='3306',
        user='root',   # MySQL用户名
        password='c0nygre', # MySQL密码
        database='project_demo'  # 目标数据库名
    )
    cursor = connection.cursor()
    query = '''
    SELECT * FROM stock_data WHERE symbol = %s 
    AND date >= %s AND date <= %s;
    '''

    cursor.execute(query, (symbol, start, end))
    data = cursor.fetchall()
    columns = [desc[0] for desc in cursor.description]
    data = pd.DataFrame(data, columns=columns)
    # print(data)

    cursor.close()
    connection.close()
    return data

# Define function to calculate the RSI
def calc_rsi(data, start, end, length=14, func_name='sma'):
    data.reset_index(inplace=True)
    data['date'] = pd.to_datetime(data['date'])
    data.set_index('date', inplace=True)

    # Get the difference in price from previous step
    data = data.loc[roll_back_days(start):end]
    delta = data['adj_close'].diff()
#     print(delta)
    # Get rid of the first row, which is NaN since it did not have a previous row to calculate the differences
#     delta = delta[1:] 

    # Make the positive gains (up) and negative gains (down) Series
    up, down = delta.clip(lower=0), delta.clip(upper=0).abs()
    
    if func_name == 'sma':
        fn_roll = lambda s: s.rolling(length).mean()
    elif func_name == 'ema':
        fn_roll = lambda s: s.ewm(span=length).mean()
    roll_up, roll_down = fn_roll(up), fn_roll(down)
    rs = roll_up / roll_down
    rsi = 100.0 - (100.0 / (1.0 + rs))

#     # Avoid division-by-zero if `roll_down` is zero
#     # This prevents inf and/or nan values.
#     rsi[:] = np.select([roll_down == 0, roll_up == 0, True], [100, 0, rsi])
#     # rsi = rsi.case_when([((roll_down == 0), 100), ((roll_up == 0), 0)])  # This alternative to np.select works only for pd.__version__ >= 2.2.0.
#     rsi.name = 'rsi'
    return rsi.loc[start:end]

def calc_moving_average(close, func_name, length=14):
    if func_name == 'sma':
        fn_roll = lambda s: s.rolling(length).mean()
    elif func_name == 'ema':
        fn_roll = lambda s: s.ewm(span=length).mean()
    return fn_roll(close)

def kline(data, start_time, end_time, symbol, func_name):
    data = data[start_time:end_time]
    data.reset_index(inplace=True)
    data['date'] = pd.to_datetime(data['date'])
    data.set_index('date', inplace=True)
#     columns_mpl = ['open', 'high', 'low', 'close', 'volume']
    sma_10 = calc_moving_average(data['close'], func_name, 10)
    print(sma_10)
    sma_30 = calc_moving_average(data['close'], func_name, 30)
    fig = plt.figure(figsize=(8,4), dpi=200)
    ax = fig.add_axes([0,0.2,1,0.5])
    ax2 = fig.add_axes([0,0,1,0.2])
#     mpf.candelstick2_ochl(ax, data['open'], data['close'], data['high'], data['low'],
#                          width=0.5, colorup='r', colordown='g', alpha=0.6)
    mpf.plot(data, type='candle', volume=ax2, ax=ax, mav=[10,30], style='yahoo')
    
#     # 获取 10 日均线和 30 日均线数据
#     mav10 = data['close'].rolling(window=10).mean()
#     mav30 = data['close'].rolling(window=30).mean()

#     # 找到均线的最后一个值
#     last_mav10 = mav10.iloc[-1]
#     last_mav30 = mav30.iloc[-1]

#     # 设置右上角的位置（可调整）
#     x_pos = data.index[-1]
#     y_pos = data['Close'].max()  # 使用最大收盘价作为 y 轴位置

#     # 添加文本到图表
#     ax.text(x_pos, y_pos, '10-Day MA', color='blue', ha='right', va='top', fontsize=10)
#     ax.text(x_pos, y_pos * 0.95, '30-Day MA', color='orange', ha='right', va='top', fontsize=10)


#     ax.plot(sma_10.index, sma_10, label='10-Day Moving Average')
#     ax.plot(sma_30.index, sma_30, label='30-Day Moving Average')
    ax.set_xticks(range(0, len(data.index), 10))
    ax.set_title(f"{symbol} Kline Chart")
    ax.legend(loc="upper left")
    ax.grid(True)
    
#     mpf.volume_overlay(ax2, data['open'], data['close'], data['volume'],
#                       colorup='r', colordown='g', width=0.5, alpha=0.8)
    ax2.set_xticks(range(0, len(data.index), 10))
    interval = len(data.index) // 12
    ax2.set_xticklabels(data.index[::interval].strftime("%Y-%m-%d"), rotation=30)
    ax2.grid(True)
    plt.show()

def rsi_chart(rsi, start, end, length=14):
    # style.use('ggplot')
    lower_bound, upper_bound = 30, 70
    red_y = np.where((rsi < lower_bound) | (rsi > upper_bound), rsi, np.nan)
    red_y = pd.Series(red_y)
    red_y.index = rsi.index
    # red_y.dropna(inplace=True)

    green_y = np.where((rsi >= lower_bound) & (rsi <= upper_bound), rsi, np.nan)
    green_y = pd.Series(green_y)
    green_y.index = rsi.index

    fig, axes = plt.subplots(figsize=(15,5))   # subplots(1,2) is one-dimension array
    axes.plot(rsi.index, rsi, color='grey', linewidth=1)
    # axes.plot(rsi.index, red_y, color='red', linewidth=1)  
    # axes.plot(rsi.index, black_y, color='black', linewidth=1)
    axes.scatter(rsi.index, red_y, color='red', s=5) 
    axes.scatter(rsi.index, green_y, color='green', s=5) 

    axes.axhline(y=lower_bound, color='lightgrey', linestyle='--', linewidth=0.8)  
    axes.axhline(y=upper_bound, color='lightgrey', linestyle='--', linewidth=0.8)  

    axes.fill_between(rsi.index, lower_bound, upper_bound, color='green', alpha=0.2)
    # axes.set_xlabel("Date")
    axes.set_ylabel("RSI")
    interval = len(rsi.index) // 10
#     axes.set_xticks(axes.get_xticks()[::interval])
#     axes.set_xticklabels(data.index[::interval].strftime("%Y-%m-%d"), rotation=30)
    axes.xaxis.set_tick_params(rotation=45)
    axes.grid(True)
    fig.show()

# Input!!!
symbol = "JD"
start, end = '2024-01-01', '2024-08-27'
length = 14
method = "sma"

stock_list = pd.read_csv('stock_list.txt', header=None)[0].tolist()
if symbol in stock_list:
    # Read from DataBase
    data = read_from_db(symbol, start, end)
else:
    data = download_from_yahoo(symbol)
data.set_index('date', inplace=True)  

rsi = calc_rsi(data, start, end, length=14, func_name=method)
kline(data, start, end, symbol, method)
rsi_chart(rsi, start, end, length=14)

ModuleNotFoundError: No module named 'fsspec.asyn'